In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
model = 'gpt-4o-mini'
db_name = 'vector_db'

In [ ]:
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
folders = glob.glob('knowledge-base/*')
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []

for folder in folders :
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata['doc_type'] = doc_type
        documents.append(doc)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

In [ ]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

In [ ]:
embeddings = OpenAIEmbeddings()

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f'Vector Store created {vectorstore._collection.count()} documents')

In [ ]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name=model)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "Please explain what Insurellm is in a couple of sentences"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

In [ ]:
def chat(message,history):
    result = conversation_chain.invoke({'question':message})
    return result['answer']

In [ ]:
view = gr.ChatInterface(fn=chat).launch()

In [ ]:
from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=model)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, memory= memory, retriever= retriever, callbacks=[StdOutCallbackHandler()])

query = 'who recieved IIOTY award in 2023?'
result = conversation_chain.invoke({'question':query})
answer = result['answer']
print('Answer: ',answer)

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name=model)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 30})
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, memory=memory, retriever=retriever)  

query = 'who recieved IIOTY award in 2023?'
result = conversation_chain.invoke({'question':query})
answer = result['answer']
print('Answer: ',answer)